In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from shapely.geometry import Point
import pandas as pd
from dotenv import dotenv_values
import logging
import folium

from generator_drainage_units.generator_order_levels import GeneratorOrderLevels

logging.basicConfig(level=logging.INFO)

In [ ]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
waterschap = "Waterschap Vallei & Veluwe"
case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
# case_name = "Leuvenumse_beek"

# Define waterboard specific variables
range_orde_code_min = 721
range_orde_code_max = 760

In [ ]:
# Load class and accompanying data as object 'case_data' for selected case


def run_generator_order_levels(
    path: Path,
    waterschap: str,
    range_orde_code_min: int,
    range_orde_code_max: int,
    read_results: bool = True,
    write_results: bool = True,
    create_html_map: bool = False,
) -> GeneratorOrderLevels:
    order = GeneratorOrderLevels(
        waterschap=waterschap,
        range_orde_code_min=range_orde_code_min,
        range_orde_code_max=range_orde_code_max,
        read_results=read_results,
        write_results=write_results,
    )
    order.read_data_from_case(path=path)

    order.find_end_points_hydroobjects()
    order.generate_rws_code_for_all_outflow_points()

    if create_html_map:
        order.generate_folium_map()
    return order

In [ ]:
case_path = Path(base_dir, case_name)
order = run_generator_order_levels(
    path=case_path,
    waterschap=waterschap,
    range_orde_code_min=range_orde_code_min,
    range_orde_code_max=range_orde_code_max,
    create_html_map=True,
)
order.folium_map

In [ ]:
order.outflow_nodes_all.to_file(Path(order.path, "inflow_outflow_points.gpkg"))